In [94]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler
from sklearn.dummy import DummyRegressor
from sklearn.utils.fixes import loguniform

import scipy.stats as stats

from ts_utils import OOSR2, modOOSR2

Train/Test split

In [95]:
df = pd.read_csv("../.data/timeseries.csv", index_col="Date")
df.index = pd.to_datetime(df.index)
df = df.round(4)

train = df.iloc[:350]
test = df.iloc[350:]

X_train, y_train = train.drop(["EXCESS_RETURN_T+1"], axis=1).values, train["EXCESS_RETURN_T+1"].values
X_test, y_test = test.drop(["EXCESS_RETURN_T+1"], axis=1).values, test["EXCESS_RETURN_T+1"].values

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((350, 49), (350,), (165, 49), (165,))

Mean model

In [96]:
naive_mean = DummyRegressor(strategy="mean")
naive_mean.fit(X_train, y_train)
modOOSR2(y_test, naive_mean.predict(X_test))

0.023305483729676668

Elastic Net

In [97]:
from sklearn.linear_model import ElasticNet

pipeline = Pipeline([
    ('scaler', "passthrough"),
    ('regressor', ElasticNet())
])

param_grid = {
    'scaler': ["passthrough", MinMaxScaler(), StandardScaler(), MaxAbsScaler()],
    'regressor__alpha': loguniform(0.001,1),
    "regressor__l1_ratio": stats.uniform(0,1)  
}

tscv = TimeSeriesSplit(n_splits=3)

grid_search = RandomizedSearchCV(
    estimator=pipeline, 
    param_distributions= param_grid,
    scoring={"modOOSR2": make_scorer(modOOSR2)},
    refit="modOOSR2", 
    cv=tscv,
    n_iter=10000, 
    n_jobs=-1
    )

grid_search.fit(X_train, y_train)

RandomizedSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                   estimator=Pipeline(steps=[('scaler', 'passthrough'),
                                             ('regressor', ElasticNet())]),
                   n_iter=10000, n_jobs=-1,
                   param_distributions={'regressor__alpha': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001E35B7DCB50>,
                                        'regressor__l1_ratio': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001E35B7DFA10>,
                                        'scaler': ['passthrough',
                                                   MinMaxScaler(),
                                                   StandardScaler(),
                                                   MaxAbsScaler()]},
                   refit='modOOSR2',
                   scoring={'modOOSR2': make_scorer(modOOSR2)})

In [98]:
results = pd.DataFrame(grid_search.cv_results_)
#results

In [99]:
grid_search.best_params_

{'regressor__alpha': 0.006061818743884995,
 'regressor__l1_ratio': 0.19768279758397134,
 'scaler': MaxAbsScaler()}

In [100]:
modOOSR2(y_test, grid_search.predict(X_test))

0.04429928708762876

In [101]:
OOSR2(y_test, grid_search.predict(X_test), naive_mean.predict(X_test))

0.021494748878206682

In [102]:
best = grid_search.best_estimator_
best.named_steps["regressor"].coef_, best.named_steps["regressor"].intercept_

(array([ 0.        , -0.        , -0.        ,  0.        , -0.        ,
        -0.01149911, -0.        ,  0.        , -0.        , -0.        ,
         0.        ,  0.        ,  0.        , -0.        ,  0.        ,
        -0.        , -0.        ,  0.        , -0.        ,  0.        ,
        -0.        ,  0.        ,  0.        , -0.        , -0.        ,
         0.        ,  0.        , -0.        , -0.        , -0.        ,
        -0.        ,  0.        , -0.        ,  0.        ,  0.        ,
         0.        , -0.        ,  0.        ,  0.        , -0.        ,
        -0.        , -0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        , -0.        ]),
 0.009029937605233046)